# Warm Start Testing

In [1]:
import numpy as np
import gurobipy as gp
import math
import sympy
import contextlib
import time
import os
from utils import result, METHODS, INF, EPS
from mps_reader_preprocessor import read_mps_preprocess
from polyhedral_model import PolyhedralModel
from polyhedron import Polyhedron

## Read in Polyhedron P

In [2]:
# problems = ['adlittle', 'afiro', 'agg2','agg3','agg', 'bandm',
#  'beaconfd', 'blend', 'bnl1', 'boeing1', 'boeing2', 'bore3d', 'brandy', 'capri',
#  'cre-c', 'cycle', 'degen2', 'degen3', 'e226', 'etamacro', 'fffff800', 'finnis',
#  'fit1d', 'fit1p', 'forplan', 'ganges', 'gfrd-pnc', 'grow15', 'grow22', 'grow7',
#  'israel', 'kb2', 'ken-07', 'lotfi', 'maros', 'modszk1', 'pds-02', 'perold',
#  'pilot4', 'pilotnov', 'qap08', 'recipe', 'sc105', 'sc205', 'sc50a',
#  'sc50b', 'scagr25', 'scagr7', 'scfxm1', 'scfxm2', 'scfxm3', 'scorpion', 'scrs8',
#  'scsd1', 'scsd6', 'scsd8', 'sctap1', 'sctap2', 'sctap3', 'seba', 'share1b',
#  'share2b', 'shell', 'ship04l', 'ship04s', 'ship08l', 'ship08s', 'ship12l', 'ship12s',
#  'sierra', 'stair', 'standata', 'standgub', 'standmps', 'stocfor1', 'stocfor2', 'tuff',
#  'vtp_base', 'wood1p']


# # mps_fn='C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\netlib_lp_subset\kb2'
# for i in problems:
#     problem_dir = 'netlib_lp_subset'
#     # problem = 'afiro'
#     mps_fn=os.path.join(problem_dir, i)
#     results_dir='results'
#     max_time=300
#     sd_method='dual_simplex'
#     reset=False,
#     partition_polytope=False
#     n=0
#     k=0
#     spindle=False
#     spindle_dim=0
#     n_cone_facets=0
#     n_parallel_facets=0

#     ### Build Initial Polyhedron from file
#     print(f'Reading {mps_fn}...')
#     c, B, d, A, b = read_mps_preprocess(mps_fn)
#     print('Building polyhedron...')
#     P = Polyhedron(B, d, A, b, c)

In [3]:
problem_dir = 'netlib_lp_subset'
# problem = 'kb2'
# problem = 'adlittle'
# problem = 'simple_ex'
problem = 'network_ex'
mps_fn=os.path.join(problem_dir, problem)
results_dir='results'
max_time=300
sd_method='dual_simplex'
reset=False,
partition_polytope=False
n=0
k=0
spindle=False
spindle_dim=0
n_cone_facets=0
n_parallel_facets=0

## Perform SDCA

### Original Code

In [4]:
### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)

Reading netlib_lp_subset\network_ex...
Building polyhedron...
Problem size: n = 5,  m_B = 10,  m_A = 4


In [5]:
print('Finding feasible solution...')
x_feasible = P.find_feasible_solution(verbose=False)

Finding feasible solution...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01


In [6]:
equality = np.all(np.isclose(np.dot(A, np.array(x_feasible)), b))
inequality = np.all(np.dot(B, np.array(x_feasible))<= d)
print(equality)
print(inequality)

True
True


In [7]:
x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

In [8]:
if c is not None:
    P.set_objective(c)

t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
# print(len(active_inds))
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}    
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)


# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
print(descent_direction)
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)
    
t3 = time.time()
sub_times['sd'].append(t3 - t2)
    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)  
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        
    t4 = time.time()
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    # compute steepest-descent direction
    pm.set_active_inds(active_inds)
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(
                                                                                                verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!

Constraints:
B_0: -1.0 x_0 + -1.0 y_pos_0 + y_neg_0 = 0.0
B_1: x_0 + -1.0 y_pos_1 + y_neg_1 = 0.0
B_2: -1.0 x_1 + -1.0 y_pos_2 + y_neg_2 = 0.0
B_3: x_1 + -1.0 y_pos_3 + y_neg_3 = 0.0
B_4: -1.0 x_2 + -1.0 y_pos_4 + y_neg_4 = 0.0
B_5: x_2 + -1.0 y_pos_5 + y_neg_5 = 0.0
B_6: -1.0 x_3 + -1.0 y_pos_6 + y_neg_6 = 0.0
B_7: x_3 + -1.0 y_pos_7 + y_neg_7 = 0.0
B_8: -1.0 x_4 + -1.0 y_pos_8 + y_neg_8 = 0.0
B_9: x_4 + -1.0 y_pos_9 + y_neg_9 = 0.0
1_norm: y_pos_0 + y_neg_0 + y_pos_1 + y_neg_1 + y_pos_2 + y_neg_2 + y_pos_3 + y_neg_3 + y_pos_4 + y_neg_4 + y_pos_5 + y_neg_5 + y_pos_6 + y_neg_6 + y_pos_7 + y_neg_7 + y_pos_8 + y_neg_8 + y_pos_9 + y_neg_9 = 1.0
A_0: x_0 + x_1 = 0.0
A_1: -1.0 x_0 + x_2 + x_3 = 0.0
A_2: -1.0 x_1 + -1.0 x_2 + x_4 = 0.0
A_3: -1.0 x_3 + -1.0 x_4 = 0.0
Polyhedral model build time: 0.0020012855529785156
after set solution: 1e+101
after set solution: 1e+101
after set sol

In [9]:
x_orig = x_current

In [10]:
obj_values[-1]

20.0

### Updated Code

In [11]:
### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)

Reading netlib_lp_subset\network_ex...
Building polyhedron...
Problem size: n = 5,  m_B = 10,  m_A = 4


In [12]:
print('Finding feasible solution...')
x_feasible = P.find_feasible_solution(verbose=False)

Finding feasible solution...


In [13]:
eq_cont_sat = np.all(np.isclose(np.dot(A, np.array(x_feasible)),b))
ineq_cont_sat = np.all(np.dot(B, np.array(x_feasible)) <=d)
print(f'equality constraints satisfied by feaisble soln: {eq_cont_sat}')
print(f'inequality constraints satisfied by feaisble soln: {ineq_cont_sat}')

equality constraints satisfied by feaisble soln: True
inequality constraints satisfied by feaisble soln: True


In [14]:
x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

In [15]:
if c is not None:
    P.set_objective(c)
 
t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
init_inds = active_inds
# print(active_inds)
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)
print(pm.x)

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}    
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)

####get edge for initial circuit direction here#########
x_feasible_2= P.second_vert(x_current, verbose=False)
# print(x_feasible_2)
edge = np.array(x_feasible_2) - np.array(x_current)
# print(edge.shape)
# init_edge_normed = np.array(P.get_normalized_circuit(np.array(x_feasible_2) - np.array(x_current)))
# print(init_edge_normed)
norm = np.linalg.norm(np.array(edge),1)
# alpha, active_inds = P.get_max_step_size(x_current, init_edge_normed, active_inds=active_inds, y_pos=None)
# print(alpha)
if norm != 0:
    init_edge = edge/(norm)
else:
    init_edge = edge
# print(init_edge.shape)
# init_edge = np.array(x_feasible_2) - np.array(x_current)
# norm = np.linalg.norm(np.array(x_feasible_2) - np.array(x_current))
# init_edge = (np.array(x_feasible_2) - np.array(x_current))/norm
# init_edge = np.array(x_feasible_2) - np.array(x_current)
print(f'first vertex: {x_feasible}')
print(f'second vertex: {x_feasible_2}')
# print(f'edge: {np.array(x_feasible_2) - np.array(x_current)}')
print(f'init edge: {init_edge}')
########################################################

# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(edge = np.array(init_edge), verbose=verbose)
print(descent_direction)
test_dir = descent_direction
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)

t3 = time.time()
sub_times['sd'].append(t3 - t2)
    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)  
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        
    t4 = time.time()
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    # compute steepest-descent direction
    pm.set_active_inds(active_inds)
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(
                                                                                                verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!

Constraints:
B_0: -1.0 x_0 + -1.0 y_pos_0 + y_neg_0 = 0.0
B_1: x_0 + -1.0 y_pos_1 + y_neg_1 = 0.0
B_2: -1.0 x_1 + -1.0 y_pos_2 + y_neg_2 = 0.0
B_3: x_1 + -1.0 y_pos_3 + y_neg_3 = 0.0
B_4: -1.0 x_2 + -1.0 y_pos_4 + y_neg_4 = 0.0
B_5: x_2 + -1.0 y_pos_5 + y_neg_5 = 0.0
B_6: -1.0 x_3 + -1.0 y_pos_6 + y_neg_6 = 0.0
B_7: x_3 + -1.0 y_pos_7 + y_neg_7 = 0.0
B_8: -1.0 x_4 + -1.0 y_pos_8 + y_neg_8 = 0.0
B_9: x_4 + -1.0 y_pos_9 + y_neg_9 = 0.0
1_norm: y_pos_0 + y_neg_0 + y_pos_1 + y_neg_1 + y_pos_2 + y_neg_2 + y_pos_3 + y_neg_3 + y_pos_4 + y_neg_4 + y_pos_5 + y_neg_5 + y_pos_6 + y_neg_6 + y_pos_7 + y_neg_7 + y_pos_8 + y_neg_8 + y_pos_9 + y_neg_9 = 1.0
A_0: x_0 + x_1 = 0.0
A_1: -1.0 x_0 + x_2 + x_3 = 0.0
A_2: -1.0 x_1 + -1.0 x_2 + x_4 = 0.0
A_3: -1.0 x_3 + -1.0 x_4 = 0.0
[<gurobi.Var x_0>, <gurobi.Var x_1>, <gurobi.Var x_2>, <gurobi.Var x_3>, <gurobi.Var x_4>]
Polyhedral model build time

IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
second_inds = P.get_active_constraints(x_feasible_2)
print(len(np.intersect1d(init_inds, second_inds)))

In [ ]:
pm.x

In [ ]:
# Bx= np.dot(B, np.array(x_feasible))
Bx_edge= np.dot(B,np.array(init_edge))
Ax= np.dot(A,np.array(init_edge))
print(Ax)
print(Bx_edge)

In [ ]:
print(A)
print(B)
print(init_edge)

In [ ]:
x_new = x_current

In [ ]:
x_orig == x_new

In [ ]:
diff = x_orig-x_new
print(diff)

In [ ]:
obj_values[-1]